<font size="10">
    <div align="center">
        Técnicas y algoritmos de búsqueda
    </div>
</font>
<font size="6">
    <div align="center">
        Informe de la práctica 3
    </div>
    <br>

</font>
<font size="5">
    <div align="center">
        Gabriel Niculescu Ruso 
    </div>
    <div align="center">
        Carlos Molera Canals
    </div>
    <div align="center">
        Jose Francisco Pérez Mompeán
    </div>
</font>
<font size="5">
    <div align="right">
        29/05/2025
    </div>
</font>


<div>
    <hi><font size="6"><strong>Index</strong></font></hi>
    <ol>
        <li><font size="4">Alpha-Beta</font></li>
        <li><font size="4">Tabla de transposición</font></li>
        <li><font size="4">Función de Evaluación</font></li>
        <li><font size="4">La red</font></li>
        <li><font size="4">Testing</font></li>
    </ol>
</div>

## **FUNCIÓN DE EVALUACIÓN**

Una vez expuestas las metodologías empleadas en términos de ejercer la búsqueda, cabe señalar cuál es la función de evaluación en base a que estas últimas se nutren y apoyan. De manera concreta, a lo largo de esta sección se explorarán los siguientes apartados componentes:

<ol>
    <li>
        <strong>Introducción sobre los parámetros</strong>
    </li>
    <li>
        <strong>Explicación de las secciones</strong>
    </li>
    <li>
        <strong>Descartes y razones</strong>
    </li>


</ol>

### **1.Introducción sobre los parámetros**

Con relación a los parámetros de que nuestra función de evaluación se compone, podemos encontras los que siguen:

**1.list_of_moves** **$\rightarrow$ list[str]**

Se expone como una lista componente en los últimos cuatro movimientos que Pacman ha desarrollado y que estarán codificados en términos de cadenas de texto en representación de los puntos cardinales. Con objeto de justificar su existencia de forma escuesta, encontramos que esta remite a la necesidad de escape frente a un bucle de movimientos que de manera posterior se explicará.<br><br>


**2.ghost_heat_map** **$\rightarrow$ dict[tuple[int,...], np.ndarray]**

Es un diccionario cuyos valores serán otros diccionarios anidados en codificación de los distintos mapas y que representarán el espectro de influencia que los fantasmas pueden tener sobre las  posiciones a ocupar sobre el layout determinado por la clave interna. <br><br>

**3.current_heat_map** **$\rightarrow$ np.ndarray**

Es un array dispuesto para la evaluación de la peligrosidad estática de las celdas libres asociadas al mapa que hemos escogido en la llamada al Agente.De esta manera pues, localizamos que junto con el parámetro anterior lo que modelaremos serán una métrica dinámica en representación de la peligrosidad relativa de las distintas posiciones que el comecocos va adoptando.<br><br>


**4.original_food** **$\rightarrow$ list[int]**

Es un array en representación del número de capsulas de comida de que cada una de las particiones del mapa es componente; partición la cual se presentará más adelante. No obstante, cabe señalar que la aplicabilidad directa de este atributo ha sido derogada en base a una serie de razones que, junto con su razón de ser a priori, serán explicadas.<br><br>

**5.state** **$\rightarrow$ GameState**

Es una representación del estado actual del juego y que nos permitirá el acceso a parámetros utilitarios tales como la posición de Pacman, la de los fantasmas; e incluso valores como la puntuación y distribución de la comida  para el ply con que trabajamos.

<div align = center>
    <img src = '../fotos/custom_evaluation_function.png'>
</div>

### **2.Explicación de las secciones**

Tal y como habíamos dejado entreveer anteriormente, existen una serie de secciones en base a las cuales se construirá el cálculo final de nuestra función de nuestra evaluación sobre el estado actual.

Cabe señalar no obstante que se omitirá la parte del código referente a la obtención de las posiciones de los fantasmas y de Pacman en tanto que de manera explícita ya ha sido mencionado anteriormente el que estas se extraigan vía el objeto GameState proporcionado como parámetro



### **SECURITY**

En esta sección lo que se buscará es calcular la distribución de peligrosidad en el mapa bajo el que trabajamos de tal forma que cada una de las casillas libres quede ponderada de base por un determinado factor de peligrosidad. 

Distinguimos dos partes claras en el proceso:<BR><br>


<u>**MAPA TOPOLÓGICO ESTÁTICO**</u>

De cara a lograr todo esto, encontramos que en primer lugar se generó un mapa de calor en representación del riesgo estático que a cada una  de las distintas zonas del mapa se asociaba.

En lo que refiere a la construcción de dicha estructura, de manera particular se atribuyó un valor de peligrosidad $k \in \left\{0,5\right\}$ a cada una de las celdas y que vino dado en base a un criterio que inequívocamente se había de apoyar sobre la información topológica. 

Con objeto de explicar cuál ha sido este criterio topológico, localizamos que ha tenido su fundamentación en el número de opciones posibles de movimiento con que Pacman contaba en cada momento. Consecuentemente, encontramos que las intersecciones entre caminos se presentarán como las zonas más seguras topológicamente, mientras que a medida que nos vamos acercando a las esquinas dicho valor de seguridad va decrementado hasta que estas últimas son alcanzadas.

No obstante, también cabe señalar cuál es el modelo de evolución de la puntuación. En primer lugar, encontramos que, tal y como hemos mencionado, a las intersecciones se les atribuirá una puntuación de peligrosidad 0 que irá incrementando a medida que nos acerquemos a los extremos del mapa hasta un valor que vendrá condicionado por la longitud de dicho recorrido y que habrá de estar en concordancia con el flujo de puntuación que llega por parte de la otra vía de  comunicación. 

Si bien puede resultar confuso, veamos el siguiente ejemplo:



<div align = center>
    <img src = '../fotos/foto_mapa_calor.png'>
</div>

Como podemos observar, el valor mínimo irá progresando e incrementando hasta que se acerque a las esquinas. No obstante, y entendiendo el mapa como un sistema recorrido por un fluido, si tenemos dos fuentes generadoras del mismo llegará un punto en que ambos se encuentren y tengan que igualar fuerzas. Cabe señalar a su vez que el avance por cada una de las distintas fuentes (círculos blancos) se  produce a la misma velocidad y en todas las direcciones que los movimientos legales de Pacman permitan en cada momento. 


En esencia, lo que se pretendía recalcar con esta aclaración es el hecho de que por cuestiones de simetría los valores mínimos de seguridad que dentro del dominio definido se encuadran no siempre van a ser alcanzables en las esquinas.


<u>**COMPONENTE DINÁMICA**</u>

Expresado lo anterior, y dado que pretendíamos ejercer una evaluación global del peligro, se presentó como necesario el considerar el impacto que la presencia de los enemigos o fantasmas podrían llegar a tener sobre la metodología de puntuación que en el primer apartado se ha establecido. 

Con ello, entenderemos al tipo fantasma como un campo con algún tipo de energía dispuesta y que a  medida que se va desplazando sobre las casillas ponderadas en base a la topología va alterando su valor. Al igual que sucede en campos definidos en términos formales, el impacto o potencia del mismo irá variando a medida que nos vayamos alejando del foco emisor. Si bien campos como el eléctrico tienen un potencial que es cero en el infinito, lo que pretendíamos modelar era más el efecto de sumidero de peligrosidad más que el ceñirnos a lo que sería correcto desde un punto de vista teórico. 

Consecuentemente, encontramos que generaremos un modelado de rango de influencia limitado $r$, centrado en el fanstasma y cuyo valor variará en voluntad de adaptarse a la geometría y dimensiones de cada uno de los mapas dispuestos. Veamos pues una pareja de ejemplos de lo que el efecto de dicho fantasma supone en mapas distintos:<br><br>


<strong>1.MAPA MEDIUM CLASSIC</strong>

<strong>2.MAPA TRICKY CLASSIC</strong>

<div align = center>
    <img src ='../fotos/output.png'>

</div>



Tal y como podemos observar, el rango de influencia o de impacto entre ambos mapas se expone como distinto del mismo modo que también lo son sus dimensiones; siendo pues este más grande en el caso del mapa de tamaño más elevado 

Finalmente, y en voluntad de exponer como dicho campo actúa sobre la evaluación estática antes ejercida, entendemos que simplemente se presentará como una combinación lineal de valores, esto es, una suma de los arrays representados en ambas metodologías de ponderación.

Sea pues esta la fórmula que codifica el valor de cada una de las celdas:<br><br>

$$\text{Final danger} = \text{Static Danger + Dynamic Danger}$$



Veamos pues un ejemplo de dicho impacto combinado para un fantasma centrado en la posición $(2,3)$ con el mapa $\text{Medium Classic}$ y su comparación con el mapa estático:


<div align = center>
    <img src ='../fotos/fotos_comparativa_estatico_dinamico.png'>

</div>

Tal y como se puede observar, en la posición de centrado en el mapa izquierdo ha incrementado la peligrosidad como producto del impacto generado por parte del fantasma.

<u>**CÓDIGO**</u>


Finalmente, cabe señalar que los diccionarios que codifican los mapas de calor estáticos y dinámicos estarán almacenados en un formato pickle para facilitar su carga.

Matizado esto último, veamos una imagen asociada al código que ha sido empleado para desempeñar la tarea:



<div align = center>
    <img src ='../fotos/codigo_security.png'>

</div>

### **MEAN-DANGER-CURRENT-QUADRANT**

A lo largo de esta sección lo que se buscará es el determinar la peligrosidad media de la división del mapa dentro de la cual Pacman se encuentra en el ply actual. 

Con ello, localizamos que los apartados construidos de cara a dichar labor son los que siguen:


<u>**DIVIDIENDO EL ESPACIO**</u>


Tal y como mencionábamos en un inicio del apartado, el mapa sufrirá una división en cuatro secciones o cuadrantes que atenderán a una partición esquitativa del espacio fundamentada bajo el trabajo con las dimensiones del layout. 

Si tomamos por ejemplo un $Layout = (x,y)$, entonces los valores en base a los cuales se van a segregar los cuadrantes serán los que siguen:

$$h = \lceil\frac{x}{2} \rceil $$
$$w = \lceil\frac{y}{2} \rceil $$




<div align = center>
    <table>
        <tr align=center>
            <td align=center> <strong>CUADRANTE<strong></td>
            <td> <strong>CONDICIÓN X<strong></td>
            <td> <strong>CONDICIÓN Y<strong></td>
        </tr>
        <tr align=center>
            <td> 1</td>
            <td> x less than h </td>
            <td> y less than w </td>
        </tr>
        <tr align=center>
            <td> 2</td>
            <td> x more equal than h </td>
            <td> y less than w </td>
        </tr>
        <tr align=center>
            <td> 3</td>
            <td> x more equal than h </td>
            <td> y more equal than w </td>
        </tr>
        <tr align=center>
            <td> 4</td>
            <td> x less than h </td>
            <td> y more equal than w </td>
        </tr>
    </table>
</div>

Una imagen asociada a dicha división representada sobre el mapa mediumClassic sería la que sigue:

<div align = center>
    <img src ='../fotos/quadrants.png'>

</div>

<u>**¿EN QUÉ CUADRANTE ESTÁ PACMAN**</u>


Construida dicha partición, habremos de determinar cuál es el cuadrante concreto en que se encuentra Pacman en voluntad de poder ejercer el cálculo sobre la media de peligrosidad de las celdas que localmente lo rodean. 

La justificación de la elección del cuadrante actual para este cálculo es que la función de evaluación adquiere un carácter estático y confiamos en la capacidad de la búsqueda para encontrar evaluaciones de Pacman en otros cuadrantes a medida que esta progresa.



<u>**MEDIA DE LA PELIGROSIDAD DEL CUANDRANTE**</u>

Simplemente se construirá como el sumatorio del riesgo dinámico asociado a cada una de las celdas que no sean muros del cuadrante en que Pacman se encuentra actualmente partido por el total de este tipo de celdas. Sea por ejemplo el cuadrante $1$ con un total de $16$ celdas que no son muros y un sumatorio de peligrosidad construido de la siguiente manera:

$$n_{celdas} = 16$$
$$danger = \sum_{i = 1}^{n_{celdas}}\text{mapa de peligrosidad}\ [i]$$

$$\text{Mean of danger}\ (\mu) = \frac{danger}{n_{celdas}}$$



<U>**CONCLUSIÓN Y VÍNCULO**</u>

En conclusión esta métrica nos permitirá aplicar la concepción de la seguridad determinada en el apartado anterior, a la par que obtener un estimador insesgado como es el de la media muestral para determinar o ponderar cómo de bueno es el cuadrante que actualmente ocupo. 

Valores muy elevados de peligrosidad media establecerán una alta voluntad de transición  hacia otros cuadrantes, mientras que términos más reducidos computarán el que se entienda el estado actual como uno no desfavorable.


### **LOOKING FOR DENSITY**


Tal y como hemos mencionado anteriormente, la consideración que ejerce la función de evaluación se desarrolla desde un punto de vista estático y por ende se tomará con respecto al cuadrante en que actualmente Pacman se encuentra.

Si bien antes hemos considerado la peligrosidad de la región donde nos hayamos, otro de los factores relevantes del juego se presenta como la cantidad de comida que somos capaces de comer en el menor tiempo posible. De ahí pues que surja la  necesidad de ponderar cuál es el punto caliente donde esta última está concentrada. 



<u>**CENTROIDE**</u>

Hilando lo descrito anteriormente, la categorización de la bondad de una posición dentro del cuadrante será, además de mi seguridad como jugador, la cercanía respecto a una mayor cantidad de cápsulas o de comida. 

Puesto que necesitábamos como tal considerar el impacto de la totalidad de las cápsulas sobre la posición actual, se decidió tomar una medida o equivalencia de la influencia que todas estas tienen en conjunto. Con ello, decidimos que el cálculo del centroide de la distribución de las cápsulas se presentaba como idóneo puesto que nos permitía considerar un punto caliente único que era media de la posición que todas las cápsulas pertenecientes a la división actual.

Sea pues la siguiente fórmula empleada para su cálculo y asumiendo la disposición de un set de comida  $S \left\{(x,y), \forall x\,y \in \N ^+ \right\}$ en el cuadrante actual:

$$\text{Centroid}\ (x_c, y_c) = \frac{\sum_{i = 0}^{|S|} S_i}{|N|} \in \N^+$$

De esta manera, disponemos de una heurística o guía que nos permite dictaminar cuál es ese punto en el cual tengo más posibilidades de alcanzar una mayor cantidad de cápsulas al desplazarse este hacia las zonas de mayor densidad.



<div align = center>
    <img src ='../fotos/CENTROID.jpg'>

</div>

<u>**DISTANCIA DE MANHATTAN**</u>

Una vez obtenido dicho valor, se computará la distancia horizontal y vertical respecto al centroide calculado de manera previa, esto es, la Distancia de Manhattan. Con ello, lo que pretenderemos penalizar es el estar alejados de ese punto caliente que sobre la distribución de comida se ha generado.

$$Manhattan(a,b) = |a_x - b_x| + |a_y - b_y|$$
Cabe señalar a su vez que en el cálculo de dicha distancia no se ha tenido en consideración la existencia de obstáculos físicos de ningún tipo y por ende la métrica calculada es un ideal que se aleja de la realidad en ese aspecto. 

En el caso de la imagen anterior por ejemplo, la distancia de Manhattan respecto al centroide se presentaría como de 2; penalización la cual se presentaría como muy reducida dada la proximidad a la comida 

### **REPEATING MOVES**



La existencia de esta sección surge en la materia que Pacman cae en ciertas ocasiones en un bucle de movimientos del tipo north-sout o left-right que repite hasta que un fantasma se encuentra lo suficientemente cerca como para romperlo.

Las causas exactas de este fenómeno no son claras ni definidas, pero tenemos las siguientes hipótesis:


1. El centroide cae en lugares centrales del cuadrante al estar distribuida la comida por los extremos del mismo y por ende no puede escapar de esa posición.

2. Si queda poca o ninguna comida en el cuadrante y pacman está alejado de otras particiones, la búsqueda no tiene la suficiente profundidad como para determinar el estado del jugador en los cuadrantes distantes. Consecuentemente, no le queda otra opción más que moverse de forma incoherente o  repetir movimientos wn bucle. 



**SOLUCIÓN**

Con voluntad de paliar esto último, y tal y como mencionábamos en la inicialización, hemos atribuido al hecho de que se repitan secuencias de movimientos una penalización de tal manera que se descarten las ramas en que estos últimos se producen.

No obstante, hay que tener cuidado con este castigo porque de ser muy severo, Pacman considerará como una opción menos mala el ser comido por un fantasma y por ende finalizar el juego. Si bien esa era la idea inicial, la aplicación práctica no ha terminado de ser fructífera en el aspecto para el cual fue creada. 



<div align = center>
    <img src ='../fotos/REPEATING_MOVS.png'>

</div>

### **LAS FUERZAS**
Si bien es una idea que finalmente ha sido descartada, se presta como interesante al menos mencionarla. 

Dada la consideración de los centroides que a los distintos cuadrantes se atribuyen como sumideros o atractores, resulta coherente el que todos ellos tiren  de Pacman con una cierta fuerza. Será la resultante de todas ellas la que actúe como una guía de la dirección o cuadrante concreto hacia el cual nos tenemos que desplazar

En términos de expresar esto último, se tomó la $\text{Ley de Gravitación Universal}$ en ausencia de la constante de Gravitación Universal y con algunas modificacinoes:

$$F = \frac{\text{Densidad comida}^{-1}}{\text{Distancia Centroide}^2}$$

Tal y como podemos observar, en lugar de ponderar el producto de las masas en el numerador, consideramos a pacman como una masa unitaria y lo ponderamos por la inversa de la densidad de comida en el cuadrante sobre el cual se está calculando la fuerza. De esta manera, lo que estamos primando es que pacman finalize con la comida en el cuadrante presente antes de que la partición más cercana y distinta de la actual tire hacia él con una mayor intensidad.


Si bien resultaba una idea interesante, han sido las dificultades de modelar algo con estricta coherencia en el numerador las que han conllevado inevitablemente a su descarte. 
